In [78]:
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from collections import OrderedDict

In [79]:
raw = pd.read_csv("all-sc-bike-data-1101.csv")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [80]:
raw.head()

,bike_id,is_disabled,is_reserved,jump_ebike_battery_level,lat,lon,name,datetime,update_time
0,bike_23366,0,0,72%,36.967843,-121.999688,1497,1540307790,NaN
1,bike_23360,0,0,54%,36.971092,-122.024257,1391,1540307790,NaN
2,bike_20799,0,0,63%,36.976315,-121.977732,958,1540307790,NaN
3,bike_21411,0,0,66%,36.952708,-122.064642,1341,1540307790,NaN
4,bike_20790,0,0,91%,36.962897,-122.045212,887,1540307790,NaN


In [81]:
raw.drop(['is_disabled', 'is_reserved', 'update_time', 'name'], axis = 1, inplace = True)

In [5]:
raw['jump_ebike_battery_level'] = raw['jump_ebike_battery_level'].str.strip('%').astype(int)

In [6]:
raw['utc_time'] = pd.to_datetime(raw['datetime'], unit='s')
raw['utc_time'] = raw['utc_time'].dt.tz_localize('UTC')
#raw['local_time'] = raw['utc_time'].dt.tz_convert('America/Los_Angeles')

In [7]:
#raw = raw.round({'lat':4, 'lon':4})

In [8]:
idle_bikes =raw.groupby(by=['bike_id', 'lat', 'lon'])

In [9]:
idle_time = idle_bikes.utc_time.max() - idle_bikes.utc_time.min()

In [10]:
idle_df = idle_time.reset_index(name='idle_time')

In [11]:
idle_df.head()

,bike_id,lat,lon,idle_time
0,bike_20631,36.9606,-122.0356,04:34:55
1,bike_20631,36.9612,-122.0448,00:37:31
2,bike_20631,36.9618,-122.0450,00:49:06
3,bike_20631,36.9620,-122.0249,00:30:07
4,bike_20631,36.9624,-122.0240,01:02:40


In [12]:
idle_batt_start = idle_bikes.jump_ebike_battery_level.first()
#idle_batt_start = idle_batt_start.reset_index(name='idle_batt_start')
idle_batt_end = idle_bikes.jump_ebike_battery_level.last()
#idle_batt_end = idle_batt_end.reset_index(name='idle_batt_end')
idle_utc_time_start = idle_bikes.utc_time.first()
#idle_utc_time_start = idle_utc_time_start.reset_index(name='idle_utc_time_start')
idle_utc_time_end = idle_bikes.utc_time.last()

In [13]:
to_merge = [idle_batt_start, idle_batt_end, idle_utc_time_start, idle_utc_time_end]
merge_names = ['batt_start', 'batt_end', 'utc_time_start', 'utc_time_end']

for m, name in zip(to_merge, merge_names):
    m_df = m.reset_index(name = name)
    idle_df = pd.merge(idle_df, m_df)
    


In [14]:
idle_df['local_time_start'] = idle_df['utc_time_start'].dt.tz_convert('America/Los_Angeles')

In [15]:
idle_df.sort_values(['bike_id', 'utc_time_start'], axis=0, inplace=True)

In [18]:
idle_df.tail(20)

,bike_id,lat,lon,idle_time,batt_start,batt_end,utc_time_start,utc_time_end,local_time_start
10480,bike_26977,36.9808,-122.0173,00:14:05,56,56,2018-10-28 19:02:49+00:00,2018-10-28 19:16:54+00:00,2018-10-28 12:02:49-07:00
10453,bike_26977,36.9640,-122.0205,00:36:34,50,50,2018-10-28 19:34:01+00:00,2018-10-28 20:10:35+00:00,2018-10-28 12:34:01-07:00
10467,bike_26977,36.9717,-122.0188,16:59:12,94,94,2018-10-29 04:00:18+00:00,2018-10-29 20:59:30+00:00,2018-10-28 21:00:18-07:00
10468,bike_26977,36.9719,-122.0188,00:00:00,94,94,2018-10-29 21:01:31+00:00,2018-10-29 21:01:31+00:00,2018-10-29 14:01:31-07:00
10464,bike_26977,36.9708,-122.0243,01:10:24,92,92,2018-10-29 21:06:34+00:00,2018-10-29 22:16:58+00:00,2018-10-29 14:06:34-07:00
10443,bike_26977,36.9598,-122.0410,00:18:07,87,87,2018-10-29 22:25:01+00:00,2018-10-29 22:43:08+00:00,2018-10-29 15:25:01-07:00
10472,bike_26977,36.9739,-122.0261,00:32:12,84,84,2018-10-29 23:08:15+00:00,2018-10-29 23:40:27+00:00,2018-10-29 16:08:15-07:00
10483,bike_26977,36.9840,-122.0293,01:12:55,81,81,2018-10-29 23:49:00+00:00,2018-10-30 01:01:55+00:00,2018-10-29 16:49:00-07:00
10482,bike_26977,36.9837,-122.0190,13:14:20,78,78,2018-10-30 01:10:26+00:00,2018-10-30 14:24:46+00:00,2018-10-29 18:10:26-07:00
10466,bike_26977,36.9717,-122.0323,01:42:42,73,73,2018-10-30 14:41:24+00:00,2018-10-30 16:24:06+00:00,2018-10-30 07:41:24-07:00


In [46]:
same_bike = idle_df.bike_id == idle_df.bike_id.shift(-1)
same_charge = idle_df.batt_end == idle_df.batt_start.shift(-1)
same_loc = (idle_df.lat.round(3) == idle_df.lat.shift(-1).round(3)) & (idle_df.lon.round(3) == idle_df.lon.shift(-1).round(3))
same_time = (idle_df.utc_time_start.shift(-1) - idle_df.utc_time_end).dt.seconds < 130
missing_idle = same_bike & same_charge & same_loc & same_time

In [47]:
missing_idle

54       False
36       False
13       False
40       False
0        False
56       False
21       False
41       False
24       False
16       False
12       False
3        False
22       False
32       False
33       False
4        False
31       False
6        False
26       False
44       False
5        False
43        True
42       False
10       False
9        False
17       False
48       False
14       False
38       False
39       False
         ...  
10476    False
10479    False
10459    False
10465    False
10474    False
10477    False
10448    False
10441    False
10442    False
10471    False
10480    False
10453    False
10467     True
10468    False
10464    False
10443    False
10472    False
10483    False
10482    False
10466    False
10475    False
10460    False
10440    False
10438    False
10478    False
10449    False
10445    False
10444    False
10470     True
10469    False
Length: 10484, dtype: bool

In [48]:
#idle_copy = idle_df.copy()

In [52]:
idle_df.utc_time_end[missing_idle] = idle_df.utc_time_end.shift(-1)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [53]:
drops = missing_idle.shift(1)
drops.iloc[0] = False
idle_df = idle_df[~drops]

In [64]:
drops.tail()

10449    False
10445    False
10444    False
10470    False
10469     True
dtype: bool

In [55]:
idle_df['idle_time'] = idle_df['utc_time_end'] - idle_df['utc_time_start']

In [76]:
charge_change = -idle_df.batt_start.diff(periods=-1)
idle_df['gets_pickedup_charged'] = charge_change > 5


In [77]:
idle_df['in_charger'] = (idle_df.batt_end - idle_df.batt_start) > 5

In [74]:
idle_df.head()

,bike_id,lat,lon,idle_time,batt_start,batt_end,utc_time_start,utc_time_end,local_time_start,gets_pickedup_charged,in_charger
54,bike_20631,36.9873,-122.0342,01:52:55,94,96,2018-10-23 19:26:39+00:00,2018-10-23 21:19:34+00:00,2018-10-23 12:26:39-07:00,False,False
36,bike_20631,36.9742,-122.0244,00:05:13,93,93,2018-10-23 21:37:55+00:00,2018-10-23 21:43:08+00:00,2018-10-23 14:37:55-07:00,False,False
13,bike_20631,36.9669,-122.0406,08:10:09,49,23,2018-10-25 01:11:07+00:00,2018-10-25 09:21:16+00:00,2018-10-24 18:11:07-07:00,False,False
40,bike_20631,36.9758,-122.0284,00:15:05,42,42,2018-10-25 03:44:59+00:00,2018-10-25 04:00:04+00:00,2018-10-24 20:44:59-07:00,False,False
0,bike_20631,36.9606,-122.0356,04:34:55,27,26,2018-10-25 04:40:17+00:00,2018-10-25 09:15:12+00:00,2018-10-24 21:40:17-07:00,True,False


In [138]:
idle_df['geolocation'] = raw.apply(lambda z: Point(z.lat, z.lon), axis=1)

In [140]:
364037.91*.0001

36.403791

In [141]:
295811.93*.0001

29.581193

In [7]:
import featurization as f

In [8]:
df = f.import_and_clean_data("all-sc-bike-data-1101.csv")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [9]:
df = f.group_and_create_target(df)

NameError: name 'get_missed_idle_bike' is not defined